In [1]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, XLMRobertaTokenizerFast, XLMRobertaXLForCausalLM
import torch

/home/wolfingten/.python_environments/ML/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/wolfingten/.python_environments/ML/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/wolfingten/.python_environments/ML/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
model = XLMRobertaXLForCausalLM.from_pretrained("xlm-roberta-base")

You are using a model of type xlm-roberta to instantiate a model of type xlm-roberta-xl. This is not supported for all configurations of models and can yield errors.
If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of XLMRobertaXLForCausalLM were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['roberta.encoder.layer.1.attention.self_attn_layer_norm.weight', 'roberta.encoder.layer.3.attention.self_attn_layer_norm.bias', 'roberta.encoder.layer.6.LayerNorm.bias', 'roberta.encoder.layer.5.attention.self_attn_layer_norm.weight', 'roberta.encoder.layer.11.attention.self_attn_layer_norm.bias', 'roberta.encoder.layer.0.LayerNorm.bias', 'roberta.encoder.layer.8.attention.self_attn_layer_norm.weight', 'roberta.encoder.LayerNorm.weight', 'roberta.encoder.layer.1.LayerNorm.weight', 'roberta.encoder.layer.5.LayerNorm.weight', 'roberta.encoder.layer.7.attention.self_attn_layer_norm.bias', 'roberta.encoder.layer.4.L

In [5]:
tokens = tokenizer("I am bald unbelievable.", return_tensors="pt")
with torch.no_grad():
    outputs = model(**tokens)


In [26]:
output_ids = tokens["input_ids"].squeeze(0)[1:-1]
index = torch.arange(0, output_ids.shape[0])
surp = -1 * torch.log2(torch.nn.functional.softmax(outputs.logits, dim=-1).squeeze(0)[index, output_ids])

In [27]:
ids = tokens.word_ids()[1:-1]

for i, surp in zip(ids, surp):
    
    print(f"{i}:{surp}")

0:44.09819030761719
1:27.66463851928711
2:30.628446578979492
3:18.88300323486328
3:22.08664321899414
3:20.027633666992188
3:32.8770866394043
3:13.566536903381348


In [30]:
import numpy as np

ids = np.array(ids)
surprisal_values = np.array(surp)

word_surprisal_sum = np.bincount(ids, weights=surprisal_values)
word_counts = np.bincount(ids)
word_surprisal_avg = word_surprisal_sum / word_counts


ValueError: object of too small depth for desired array

In [33]:
output_ids = tokens["input_ids"].squeeze(0)[1:]

tensor([   87,   444, 33984,     5,     2])

In [27]:
outputs.

tensor([[[ 1.4304e+01, -6.9474e-02,  2.8266e+01,  ...,  1.4371e+01,
           4.7493e+00,  1.1140e+01],
         [ 1.0702e+01,  3.4034e-03,  2.1005e+01,  ...,  1.6348e+01,
           6.6278e+00,  9.6904e+00],
         [ 1.0795e+01,  4.1587e-02,  2.0379e+01,  ...,  1.5365e+01,
           5.9377e+00,  9.3574e+00],
         [ 1.1029e+01,  1.0021e-01,  2.0498e+01,  ...,  1.6095e+01,
           6.1641e+00,  9.0450e+00],
         [ 8.6666e+00,  3.4548e-02,  2.0787e+01,  ...,  1.5092e+01,
           6.0019e+00,  9.5058e+00],
         [ 1.2237e+01,  2.2618e-02,  2.0028e+01,  ...,  1.5195e+01,
           5.9527e+00,  9.4392e+00]]])